In [1]:
import os
import json
from openai import OpenAI
import urllib.parse
from amadeus import Client
from amadeus import ResponseError

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [25]:
amadeus = Client(
    client_id=os.getenv("AMADEUS_CLIENT_ID"),
    client_secret=os.getenv("AMADEUS_CLIENT_SECRET")
)

In [27]:
MODEL = "gpt-4o-mini"

In [29]:
def generate_itinerary(destination, days, interests, budget=None):
    prompt = f"""
    You are a travel planner. Create a {days}-day itinerary for {destination}.
    User interests: {interests}.
    Budget: {budget if budget else "not specified"}.

    Include:
    - Morning, Afternoon, Evening activities
    - One restaurant recommendation per day
    - One backup indoor activity

    Format strictly as JSON:
    {{
      "title": "Trip to {destination}",
      "summary": "short overview",
      "days": [
        {{
          "day": 1,
          "activities": ["Morning: ...", "Afternoon: ...", "Evening: ..."],
          "restaurant": "restaurant name",
          "backup_activity": "indoor option"
        }}
      ]
    }}
    """

    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful travel assistant that outputs JSON only."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=800,
        temperature=0.7
    )

    text = response.choices[0].message.content.strip()
    try:
        return json.loads(text)
    except:
        return {"raw_response": text}


In [31]:
def generate_search_links(destination):
    """
    Generate links for flights, hotels, and restaurants without APIs.
    """
    query = urllib.parse.quote(destination)
    links = {
        "flights": f"https://www.google.com/flights?hl=en#flt=/m/0{query}",
        "hotels": f"https://www.booking.com/searchresults.html?ss={query}",
        "restaurants": f"https://www.google.com/maps/search/restaurants+in+{query}"
    }
    return links

In [33]:
if __name__ == "__main__":
    destination = input("Enter destination: ")
    days = int(input("Number of days? "))
    interests = input("Your interests (e.g., beaches, museums, food)? ")
    budget = input("Budget (low, moderate, luxury)? ")

    itinerary = generate_itinerary(destination, days, interests, budget)
    search_links = generate_search_links(destination)

    print("\n🗓️ AI-generated Itinerary:\n")
    print(json.dumps(itinerary, indent=2, ensure_ascii=False))

    print("\n🌐 Suggested Flight & Hotel Links:")
    for key, link in search_links.items():
        print(f"{key.title()}: {link}")

Enter destination:  europe
Number of days?  4
Your interests (e.g., beaches, museums, food)?  beaches
Budget (low, moderate, luxury)?  luxury



🗓️ AI-generated Itinerary:

{
  "title": "Trip to Europe",
  "summary": "A luxurious 4-day itinerary exploring the beautiful beaches of Europe, combining relaxation, gourmet dining, and cultural experiences.",
  "days": [
    {
      "day": 1,
      "activities": [
        "Morning: Arrive in Nice, France. Relax on the stunning Promenade des Anglais and enjoy the Mediterranean views.",
        "Afternoon: Spend time at Plage Beau Rivage, where you can sunbathe on a private beach and enjoy water sports.",
        "Evening: Stroll through the Old Town (Vieux Nice) and dine at Le Chantecler, a Michelin-starred restaurant."
      ],
      "restaurant": "Le Chantecler",
      "backup_activity": "Visit the Musée Matisse to explore the artist's works."
    },
    {
      "day": 2,
      "activities": [
        "Morning: Take a private yacht trip to the nearby Lérins Islands for a morning of swimming and sunbathing.",
        "Afternoon: Enjoy a gourmet picnic on the beach at Île Sainte-Margu